In [ ]:
#load the data

import csv
import pandas as pd
import numpy as np
import scipy as sp 
import math



msa[[0,1]] = msa[0].str.split(' ', expand=True)
#msa[0] = pd.to_numeric(msa[0])
Q = np.array([[-0.5625,0.1875,0.1875,0.1875],[0.1875,-0.5625,0.1875,0.1875],
 [0.1875,0.1875,-0.5625,0.1875],[0.1875,0.1875,0.1875,-0.5625]])

for i in table[0]:
    if str(i) not in list(table[1]):
        print(i)

226
226


In [163]:
#load the data

import csv
import pandas as pd
import numpy as np
import scipy as sp 
import math
with open('table.dat', 'r') as file:
    table2 = pd.read_csv(file, delimiter=',', header = None)

with open('msa.dat', 'r') as file:
    msa2 = pd.read_csv(file, delimiter=',', header = None)

with open('branchlength.dat', 'r') as file:
    brlen2 = pd.read_csv(file, delimiter=',', header = None)



msa2[[0,1]] = msa2[0].str.split(' ', expand=True)
#msa[0] = pd.to_numeric(msa[0])
Q = np.array([[-0.5625,0.1875,0.1875,0.1875],[0.1875,-0.5625,0.1875,0.1875],
 [0.1875,0.1875,-0.5625,0.1875],[0.1875,0.1875,0.1875,-0.5625]])

for i in table2[0]:
    if str(i) not in list(table2[1]):
        print(i)

9
9
8
8
7
7
6
6


In [169]:
def create_node_list(tab, ms, brl):
    node_list = []
    nodes = {}
    branches = iter(brl.iloc[0])
    for i in range(len(tab[1])):
            
        itself = str(tab[1][i])
        parent =  tab[0][i]
        sib_index = [index for index, value in enumerate(tab[0]) if value == parent and str(tab[1][index]) != itself]
        sibling = str(tab[1][sib_index[0]])
        branch = next(branches)
        sequence = np.array(list(ms.loc[ms[0] == itself, 1].iloc[0])) if itself in ms[0].values else None
        root = False       
        nodes[itself] = Node(itself, str(parent), sibling, sequence, branch, root)
        node_list.append(nodes[itself])
    for i in tab[0]:

        if str(i) not in [str(x) for x in list(tab[1])]:
            itself = str(i)
            parent =  None
            sibling = None
            branch = None
            root = True
            sequence = np.array(list(ms[1][itself])) if itself in ms[0] else None
            nodes[itself] = Node(itself, parent, sibling, sequence, branch, root)
            node_list.append(nodes[itself])
            break
    return node_list

def find_node_by_number(node_list, number):
    for node in node_list:
        if node.number == number:
            return node
    return None

def find_node_by_sigling(node_list, number):
    for node in node_list:
        if node.number == find_node_by_number(node_list, number).sibling and node.number is not None:
            return node
    return None
def find_node_by_parent(node_list, number):
    for node in node_list:
        if node.number == find_node_by_number(node_list, number).parent:
            return node
    return None
def find_root_node(node_list):
    for node in node_list:
        if node.root == True:
            return node
    return None

In [165]:
#define the class

class Node:
    def __init__(self, number, parent, sibling, seq, length, root):
        self.number = number
        self.parent = parent
        self.sibling = sibling
        self.seq = seq
        self.length = length
        self.root = root

    def add_probability_matrix(self, matrix):
        if hasattr(self, 'length') and self.length is not None:
            self.matrix = sp.linalg.expm(self.length * matrix)
        else:
           # print("No length attribute found")
            self.matrix = None
    def add_seq_probability(self):
        base_prob = {"A": np.array([1,0,0,0]), "T":np.array([0,1,0,0]), "C" : np.array([0,0,1,0]), "G" : np.array([0,0,0,1])}
        map_values = np.vectorize(lambda x: base_prob[x], signature='()->(n)')
        if hasattr(self, 'seq') and self.seq is not None:
            self.prob = map_values(self.seq)
        else:
            #print("No sequence attribute found")
            self.prob = None

class Tree:
    def __init__(self, position):
        self.position = position
    def fill_probability_matrix(self, matrix):
        for node in self.position:
            node.add_probability_matrix(matrix)
            node.add_seq_probability()
    def fill_probability_sequence(self):
        root_node = find_root_node(self.position)
        b = len(self.position)
        a = 0
        while a <= b:
            for node in self.position:
                sib = find_node_by_sigling(self.position, node.number)
                parent = find_node_by_parent(self.position, node.number)
                if sib == None or parent == None:
                    continue
                if  node.root == False and node.prob is not None and sib.prob is not None and parent.prob is None:
                    vec1 = np.matmul(node.prob, node.matrix)
                    vec2 = np.matmul(sib.prob, sib.matrix)
                    vec_parent = vec1 * vec2
                    parent.prob = vec_parent
                else:
                    continue
            if root_node.prob is not None:
                break
            a += 1

    def tree_log_likelihood(self, matrix):
        self.fill_probability_matrix(matrix)
        self.fill_probability_sequence()
        root_node = find_root_node(self.position)
        npmm = np.matmul(root_node.prob, np.array([0.25,0.25,0.25,0.25])) 
        log_likelihood = sum([math.log(npmm.flatten()[i]) for i in range(len(npmm.flatten()))])
        return log_likelihood
        

In [171]:
tree_test = Tree(create_node_list(table, msa, brlen))
tree_test2 = Tree(create_node_list(table2, msa2, brlen2))
#tree_test.fill_probability_matrix(Q)
#tree_test.fill_probability_sequence()
#print([node.root for node in tree_test.position])
#print(find_root_node(tree_test.position).prob)
#tree_test.fill_probability_sequence()
#print(len([node.matrix for node in tree_test.position if node.matrix is not None]))
#print(len([node.seq for node in tree_test.position if node.seq is not None]))
#print(len(tree_test.position))
#print(find_node_by_sigling(tree_test.position, 230)) 
#print(find_node_by_number(tree_test.position, '230'))
#print(len([node for node in tree_test.position if node.seq is not None and find_node_by_sigling(tree_test.position, node).seq is None]))
#tree_test.tree_log_likelihood(Q)
#for node in tree_test.position:
 #   print(find_node_by_sigling(tree_test.position, node.number).sibling)
#print(find_root_node(tree_test.position).number)
#print(find_node_by_sigling(tree_test.position, tree_test.position[228].number).number)
#print(find_node_by_sigling(tree_test.position, tree_test.position[228].number).number)
#for node in tree_test.position:
 #   print(find_node_by_sigling(tree_test.position, node.parent))
print(tree_test.tree_log_likelihood(Q)) 
print(tree_test2.tree_log_likelihood(Q))



-7827.88183282044
-188.20007684036113
